In [6]:
import requests
import json

# URL of the API with the parameters we want
url = "https://apidatos.ree.es/es/datos/generacion/estructura-generacion"
params = {
    "start_date": "2014-01-01T00:00",
    "end_date": "2018-12-31T23:59",
    "time_trunc": "year",
    "geo_trunc":"electric_system",
    "geo_limit":"ccaa",
    "geo_ids":"7"
}

# Send request GET
response = requests.get(url, params=params)

# Check if it worked
if response.status_code == 200:
    data = response.json()
    # Print it in JSON
    print(json.dumps(data, indent=2, ensure_ascii=False))
else:
    print(f"Erreur {response.status_code}")
    print(response.text)  # Montre le message d'erreur complet


Erreur 500
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8" />
    <meta name="robots" content="noindex,nofollow,noarchive" />
    <title>An Error Occurred: Internal Server Error</title>
    <style>body { background-color: #fff; color: #222; font: 16px/1.5 -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif; margin: 0; }
.container { margin: 30px; max-width: 600px; }
h1 { color: #dc3545; font-size: 24px; }
h2 { font-size: 18px; }</style>
</head>
<body>
<div class="container">
    <h1>Oops! An Error Occurred</h1>
    <h2>The server returned a "500 Internal Server Error".</h2>

    <p>
        Something is broken. Please let us know what you were doing when this error occurred.
        We will fix it as soon as possible. Sorry for any inconvenience caused.
    </p>
</div>
</body>
</html>


In [28]:
import requests, json, datetime as dt, pandas as pd

# ───────────────────────────────────────────────────────────────────────────
# 1. Dictionnaire REGIONES  →  clé lisible  →  id API
# ───────────────────────────────────────────────────────────────────────────
REGIONES = {
    "Andalucía": 4,
    "Aragón": 5,
    "Cantabria": 6,
    "Asturias": 11,
    "Castilla y León": 8,
    "Castilla-La Mancha": 7,
    "Cataluña": 9,
    "Comunidad Valenciana": 15,
    "Extremadura": 16,
    "Galicia": 17,
    "Madrid": 8752,
    "Murcia": 21,
    "Navarra": 14,
    "País Vasco": 10,
    "La Rioja": 20,
    "Islas Baleares": 8743,
    "Islas Canarias": 8742,
    "Ceuta": 8744,
    "Melilla": 8745,
    "Península": 8741,
}

# ───────────────────────────────────────────────────────────────────────────
# 2. Choix interactifs   (région + dates)
# ───────────────────────────────────────────────────────────────────────────
print("╔══════════════════════════════════════════╗")
print("║   Régions disponibles (geo_limit=ccaa)   ║")
print("╚══════════════════════════════════════════╝")
for n, reg in enumerate(REGIONES, 1):
    print(f"{n:>2}. {reg}")

# --- sélection de la région ---
while True:
    try:
        idx = int(input("\nNuméro de la région désirée : "))
        region_name = list(REGIONES)[idx - 1]
        geo_id = REGIONES[region_name]
        break
    except (ValueError, IndexError):
        print("⛔ Choix invalide, réessaye…")

# --- saisie des dates ---
def ask_date(prompt):
    while True:
        try:
            txt = input(prompt)
            return dt.datetime.strptime(txt.strip(), "%Y-%m-%d %H:%M")
        except ValueError:
            print("⛔ Format invalide. Exemple : 2019-01-01 00:00")

start = ask_date("\nDate de début  (AAAA-MM-JJ HH:MM) : ")
end   = ask_date("Date de fin    (AAAA-MM-JJ HH:MM) : ")
if end <= start:
    raise ValueError("La date de fin doit être postérieure à la date de début.")

print(f"\n▶ Région : {region_name}  (geo_id = {geo_id})")
print(f"▶ Période : {start}  →  {end}\n")

# ───────────────────────────────────────────────────────────────────────────
# 3. Générateur de fenêtres ≤ 1 an
# ───────────────────────────────────────────────────────────────────────────
def yearly_windows(a, b):
    cur = a
    while cur < b:
        nxt = cur.replace(year=cur.year + 1) - dt.timedelta(minutes=1)
        if nxt > b:
            nxt = b
        yield cur, nxt
        cur = nxt + dt.timedelta(minutes=1)

# ───────────────────────────────────────────────────────────────────────────
# 4. Fonction d’appel API  (granularité horaire)
# ───────────────────────────────────────────────────────────────────────────
URL     = "https://apidatos.ree.es/es/datos/generacion/estructura-generacion"
HEADERS = {"Accept": "application/json"}

def fetch_gen(geo_id, a, b):
    params = {
        "start_date": a.strftime("%Y-%m-%dT%H:%M"),
        "end_date":   b.strftime("%Y-%m-%dT%H:%M"),
        "time_trunc": "hour",
        "geo_trunc":  "electric_system",
        "geo_limit":  "ccaa",
        "geo_ids":    f"{geo_id}"
    }
    r = requests.get(URL, params=params, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()["included"]        # adapte si la structure change

# ───────────────────────────────────────────────────────────────────────────
# 5. Boucle sur les fenêtres, concaténation des résultats
# ───────────────────────────────────────────────────────────────────────────
rows = []
for a, b in yearly_windows(start, end):
    print(f"  ↳ Téléchargement {a:%Y-%m-%d} → {b:%Y-%m-%d} …")
    rows.extend(fetch_gen(geo_id, a, b))

print(f"\n✅ Total reçu : {len(rows):,} lignes horaires")

df = pd.json_normalize(rows)
print(df.head())

# ── Option : export CSV / Parquet ──────────────────────────────────────────
# df.to_csv(f"estructura_{region_name}_{start:%Y%m%d}_{end:%Y%m%d}.csv", index=False)


╔══════════════════════════════════════════╗
║   Régions disponibles (geo_limit=ccaa)   ║
╚══════════════════════════════════════════╝
 1. Andalucía
 2. Aragón
 3. Cantabria
 4. Asturias
 5. Castilla y León
 6. Castilla-La Mancha
 7. Cataluña
 8. Comunidad Valenciana
 9. Extremadura
10. Galicia
11. Madrid
12. Murcia
13. Navarra
14. País Vasco
15. La Rioja
16. Islas Baleares
17. Islas Canarias
18. Ceuta
19. Melilla
20. Península

Numéro de la région désirée : 1

Date de début  (AAAA-MM-JJ HH:MM) : 2015-10-10 10:00
Date de fin    (AAAA-MM-JJ HH:MM) : 2015-10-11 10:00

▶ Région : Andalucía  (geo_id = 4)
▶ Période : 2015-10-10 10:00:00  →  2015-10-11 10:00:00

  ↳ Téléchargement 2015-10-10 → 2015-10-11 …


HTTPError: 500 Server Error: Internal Server Error for url: https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2015-10-10T10%3A00&end_date=2015-10-11T10%3A00&time_trunc=hour&geo_trunc=electric_system&geo_limit=ccaa&geo_ids=4